In [12]:
!pip3 install -r requirements.txt


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: pip3 install --upgrade pip


In [13]:
import os
from pathlib import Path

import dotenv
from openai import OpenAI

from media_extractor import split_video
import datauri

In [15]:
dotenv.load_dotenv()
if os.environ.get("OPENAI_API_KEY") is None:
    raise ValueError("OPENAI_API_KEY not found in .env file")

client = OpenAI()

In [24]:
video_file = "input1.mov"

from IPython.display import Video
Video(video_file, width=320)

In [25]:
audio_uri, image_uris = split_video(video_file)
audio_uri[:50]

'data:audio/mpeg;base64,SUQzBAAAAAAAe1RYWFgAAAASAAA'

In [26]:
with datauri.as_tempfile(audio_uri) as audio_file:
    transcription = client.audio.transcriptions.create(
        model="whisper-1", file=Path(audio_file)
    )
    
user_prompt = transcription.text
user_prompt

'Hi! What do you think of my hair and my shirt today?'

In [27]:
response = client.chat.completions.create(
    model="gpt-4o",
    messages = [
        {
            "role": "user",
            "content": [
                {"type": "text", "text": user_prompt},
                *[
                    {
                        "type": "image_url",
                        "image_url": {"url": image_uri, "detail": "auto"},
                    }
                    for image_uri in image_uris
                ],
            ],
        },
        {
            "role": "system",
            "content": Path("system_prompt.txt").read_text(),
        },
    ],
)
response_text = response.choices[0].message.content
response_text

'Hey! I think your shirt looks super cool with that space theme and colorful design; "Wayfare Through Time" sounds really intriguing! Your hair looks nice and neat in a bun, and those glasses definitely add a stylish touch. Looks like you\'re all set for a great day!'

In [28]:
audio = client.audio.speech.create(
    model="tts-1",
    voice="nova",
    input=response_text,
    response_format="mp3",
)
response_audio_uri = datauri.from_bytes(audio.read(), "audio/mpeg")

In [29]:
with datauri.as_tempfile(response_audio_uri) as response_audio_file:
    from IPython.display import Audio
    display(Audio(response_audio_file))